In [18]:
import Pkg
Pkg.add("SDDP")
Pkg.add("JuMP")
Pkg.add("HiGHS")

   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
   Installed HiGHS ───── v1.9.0
   Installed HiGHS_jll ─ v1.7.0+0
    Updating `C:\Users\matsz\.julia\environments\v1.10\Project.toml`
  [87dc4568] + HiGHS v1.9.0
    Updating `C:\Users\matsz\.julia\environments\v1.10\Manifest.toml`
  [87dc4568] + HiGHS v1.9.0
  [8fd58aa0] + HiGHS_jll v1.7.0+0
Precompiling project...
  ✓ HiGHS_jll
  ✓ HiGHS
  2 dependencies successfully precompiled in 16 seconds. 406 already precompiled.


In [19]:
using SDDP, JuMP, HiGHS

In [35]:
function asset_management_simple()
    model = SDDP.LinearPolicyGraph(
        stages = 3,
        sense = :Min,
        lower_bound = -1_000.0,
        optimizer = HiGHS.Optimizer,
    ) do subproblem, stage
        returns = [0.866, 1.06, 1.254]
        probability = [.28 .44 .28]
        r_bonds = .03
        @variable(subproblem, stocks >= 0, SDDP.State, initial_value = 0.0)
        @variable(subproblem, bonds >= 0, SDDP.State, initial_value = 0.0)
        @variable(subproblem, consumption >= 0)
        @variable(subproblem, r_stocks >= 0)

        if stage == 1
            @constraint(subproblem, stocks.out + bonds.out +consumption == 1)
            @stageobjective(subproblem, -consumption)

        elseif 1 < stage < 4
            @constraint(
                subproblem,
                r_stocks * stocks.in + r_bonds * bonds.in == stocks.out + bonds.out + consumption
            )
            @stageobjective(subproblem, -consumption)
             ### todo
        else
            @variable(subproblem, wealth >= 0, initial_value = 0.0)
            @constraint(
                subproblem,
                r_stocks * stocks.in +r_bonds * bonds.in == wealth
            )
            @stageobjective(subproblem, -wealth)

            SDDP.parameterize(subproblem, returns, probability) do w
              JUMP.fix(r_stocks,w)
            end
        end
        #end todo
    end
    SDDP.train(model; log_frequency = 5)
    #@test SDDP.calculate_bound(model) ≈ 1.514 atol = 1e-4
    return
end

asset_management_simple()

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 3
  state variables : 2
  scenarios       : 1.00000e+00
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [7, 7]
  AffExpr in MOI.EqualTo{Float64}         : [1, 1]
  QuadExpr in MOI.EqualTo{Float64}        : [1, 1]
  VariableRef in MOI.GreaterThan{Float64} : [5, 5]
  VariableRef in MOI.LessThan{Float64}    : [1, 1]
numerical stability report
  matrix range     [1e+00, 1e+00]
  objective range  [1e+00, 1e+00]
  bounds range     [1e+03, 1e+03]
  rhs range        [1e+00, 1e+00]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  

MathOptInterface.UnsupportedConstraint{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.EqualTo{Float64}}: MathOptInterface.UnsupportedConstraint{MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.EqualTo{Float64}}: `MathOptInterface.ScalarQuadraticFunction{Float64}`-in-`MathOptInterface.EqualTo{Float64}` constraint is not supported by the model.